## Implement unpadded message recovery oracle

Nate Lawson says we should stop calling it "RSA padding" and start calling it "RSA armoring". Here's why.

Imagine a web application, again with the Javascript encryption, taking RSA-encrypted messages which (again: Javascript) aren't padded before encryption at all.

You can submit an arbitrary RSA blob and the server will return plaintext. But you can't submit the same message twice: let's say the server keeps hashes of previous messages for some liveness interval, and that the message has an embedded timestamp:

```json
{
  time: 1356304276,
  social: '555-55-5555',
}
```

You'd like to capture other people's messages and use the server to decrypt them. But when you try, the server takes the hash of the ciphertext and uses it to reject the request. Any bit you flip in the ciphertext irrevocably scrambles the decryption.

This turns out to be trivially breakable: 

* Capture the ciphertext C
* Let N and E be the public modulus and exponent respectively
* Let S be a random number > 1 mod N. Doesn't matter what.
* Now:

```
C' = ((S**E mod N) C) mod N
```

* Submit C', which appears totally different from C, to the server, recovering P', which appears totally different from P
Now:

```
          P'
    P = -----  mod N
          S
```

 Oops!

Implement that attack.
### Careful about division in cyclic groups.
Remember: you don't simply divide mod N; you multiply by the multiplicative inverse mod N. So you'll need a modinv() function.




### One-time setup
Configure the server's RSA keypair and textbook RSA decrypt function.

In [1]:
from Crypto.Util.number import getPrime

In [2]:
NUM_BITS = 1024
p = getPrime(NUM_BITS)
q = getPrime(NUM_BITS)

In [3]:
n = p * q
n.bit_length()

2047

In [5]:
phi = (p - 1) * (q - 1) # Euler's totient.

In [7]:
d = pow(e, -1, phi)
d

9999084168040618735634861551028646714742419824776844939618046517930489960096612274105788204465626306989470696783698343279280014861995265035569539280891631064297939780160124628869695269343557975135982891073329303818009473603698633092090004042698768900183665105420632261310775144887374464668624168201333009266899950812475295837304653006366264235873345873988264006742306399022556182903044439835865635144643674405954517323848976805487216372176489900871366907034146005163955964589454433788255139460061307379840363093156441943600475313978033317975884382581433172679113666070300725777779503999992424431816057691268724076147

In [9]:
assert e * d % phi == 1

### Enter notebook here for re-runs.

In [2]:
N = 14998626252060928103452292326542970072113629737165267409427069776895734940144918411158682306698439460484206045175547514918920022292992897553354308921337446596446909670240186943304542904015336962703974336609993955727014210405547949638135006064048153350275497658130948391966162717331061697002936252301999513900597824741726854402436970305499076189535485327631912218072518428911792985143753609764640004919486259062931216650284588585901673444541101407504191882322230182874555079901080636451642823862690883410812084807478665973587954142541349197747345526288869524033493400650480652831708745653869356203205959909973122017029
e = 3 # public exponent.

In [3]:
from Crypto.Util.number import bytes_to_long, long_to_bytes

In [4]:
m = b"retreat"

In [5]:
C = pow(bytes_to_long(m), e, N)

In [6]:
def decrypt(c):
    N = 14998626252060928103452292326542970072113629737165267409427069776895734940144918411158682306698439460484206045175547514918920022292992897553354308921337446596446909670240186943304542904015336962703974336609993955727014210405547949638135006064048153350275497658130948391966162717331061697002936252301999513900597824741726854402436970305499076189535485327631912218072518428911792985143753609764640004919486259062931216650284588585901673444541101407504191882322230182874555079901080636451642823862690883410812084807478665973587954142541349197747345526288869524033493400650480652831708745653869356203205959909973122017029
    d = 9999084168040618735634861551028646714742419824776844939618046517930489960096612274105788204465626306989470696783698343279280014861995265035569539280891631064297939780160124628869695269343557975135982891073329303818009473603698633092090004042698768900183665105420632261310775144887374464668624168201333009266899950812475295837304653006366264235873345873988264006742306399022556182903044439835865635144643674405954517323848976805487216372176489900871366907034146005163955964589454433788255139460061307379840363093156441943600475313978033317975884382581433172679113666070300725777779503999992424431816057691268724076147
    return long_to_bytes(pow(c, d, N))

Test the "oracle".

In [7]:
decrypt(C)

b'retreat'

### Try attack
Capture ciphertext $C$.

In [8]:
from secrets import randbelow

Let $S$ be a random number > 1 mod N.

Technically, calling `randbelow` could return 0, but this should suffice for our purposes.

In [9]:
S = randbelow(N)
assert S < N
S

13485922179403841019777644665192115435919012971434682042388978589234564259170809784993802577663696714608831209358008752973547778377403289874834988353747932476765360042443885704412544085265949603883600445005323271065664810142950419484330779511850432998977786995355350372579735697623088108095054433029258720316684665682359966738307863545974928945391252210406432254242622982164162577076575681634874376378699595052428157944090367210459266236331081136402879953598417752779290757549769467532210811152948261759684458071754406046677806576442013034546170156372298438284490354407490053855106068984782683932263767480851948231080

$C' = S^e \cdot C\pmod{N}$

In [10]:
C_ = pow(S, e, N) * C % N
C_

14810958663465419568102088467393898384020101848219112058469364916554411820565161185709248254689480476584076872767788695022004480041706309311377754818066938798021797365561378172677525785575733596754971722081575076399887370229278053963244920311792839012394385273981507447040543773486080323575103564107139619360529404690980849303840055155301206940037937912170371533639550666601706114722150043199014743996249669063222605884729751161352168330365662256752636438203497304350299794969652840218753217468346585015772638822035927703598282835879830497149934918103509318544674975434556459757549514230869201088208575746380470216519

Submit $C'$ to oracle, which returns $P'$.

In [11]:
P_ = decrypt(C_)
P_

b'\x03$\xdf\x9b\x95\x8f\'>g9\x9a=\x02J\xd3`\xba\xd1\xa8\xc4;\x16\xe3]\x16\x91dy\x02\x1c=\xad\x85\xd6\x0e\xcf\x13\xa8C\xed\x9ap\x12ET"J\xadX\xb3\xcd\xeb\xf0\xb8i\xaeD\x80\x1c\n\xe3A\xd0n\x13%x\xb6&x\xe2\xc59I6\xfdNr\xd26\xaf\x9a\x17\xbe\xa5,\x00d\xf7\x1c\xa7;\x0f\x97\x17a\x08\x1b+\xa2\xa9\xf6\n\xe0\xbc+\xf0\xa5\xe2)]j.l>5l2o\x04Yol>Z\x9c#\x98\xcf\x14\x0f\xd1j]@\x98j\x8c\x9d\xd4\xc4\xf9\xee\x9aT\x96B\xb6\xb8\xa1\x0e\xe7$\x89\xe1\xec\x1a\xbeZ(\x97Lq\xc0\x9bC!f\x08\r\xaf\xd20U=\xdd\xf4.G-9C\xd5]-t\x89\x84\xba6:K\xfa\xa9\xb9\xbea\xe0\xbaZ\xb9\xbd\xfcP\xed\xb8u\xcbl\x93h\x14\x95\xd4\x7f+\xde\x86y\xa0\x92$8v\xd2\xbcB\xfc\x10qO\x88\xd4\x83\\*_\xc3\xe1\xe6\x8f\'\xc3\x05\xe0M&\xc6\xfa;V\x19\xcc\x87s\x08'

$p = \frac{P'}{S}\pmod{N}$

In [13]:
p = bytes_to_long(P_) * pow(S, -1, N) % N
p = long_to_bytes(p)
p

b'retreat'

In [14]:
assert p == m

$P' = C'^d\pmod{N} = (S^e\cdot C)^d\pmod{N} = S^{ed}\cdot C^d\pmod{N} = S\cdot m^{ed}\pmod{N} = S\cdot m\pmod{N}$

Compare math to *blinding attack* described in Aumasson (2018, page 189).